# Deep Learning Project

In [ ]:
%pip install ftfy regex tqdm ultralytics optuna
%pip install git+https://github.com/openai/CLIP.git

In [ ]:
from typing import Any, Dict, List, Tuple, OrderedDict, Callable
from dataclasses import dataclass
from enum import Enum
import pickle
import json
import gdown
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch import Tensor, tensor, device
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import torchvision.transforms as T
from torchvision.ops import box_iou, box_convert, generalized_box_iou_loss
from torchvision.io import read_image

from clip import clip
from clip.model import CLIP, ModifiedResNet

import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
import optuna
from optuna.trial import Trial
from optuna.visualization import plot_optimization_history

In [ ]:
# Download dataset and save under data/raw/ only if not already downloaded
url = "https://drive.google.com/uc?id=1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq"
if not os.path.exists("data/refcocog.tar.gz"):
    print("Downloading dataset...")
    gdown.download(url=url, output="data/", quiet=False, resume=True)
if not os.path.exists("data/refcocog/"):
    print("Extracting dataset...")
    !tar -xf data/refcocog.tar.gz -C data/ --verbose

The preprocessed samples can be downloaded from Google Drive by executing the following cell. Otherwise, the preprocessing wil be done saving the file only temporarly in the Colab environment. 

In [ ]:
# Download preprocessed dataset
url = "https://drive.google.com/drive/folders/1jaJV40dneOckZn7WHMQyd2jBh7A8534N"
gdown.download_folder(url=url, output="data/", quiet=False)

Utility classes definition

In [ ]:
@dataclass(frozen=True)
class Sample:
    image_path: str
    caption: str
    bounding_box: Tensor


class BatchSample:
    def __init__(self, image: Tensor, caption: Tensor) -> None:
        self.image: Tensor = image
        self.caption: Tensor = caption

    def to(self, device: device | str) -> Any:
        return self.__class__(self.image.to(device), self.caption.to(device))

    def __str__(self) -> str:
        return f"BatchSample(image={self.image.shape}, caption={self.caption.shape})"


@dataclass(frozen=True)
class Split(Enum):
    TRAIN = "train"
    VAL = "val"
    TEST = "test"


# Used in the baseline implementation
@dataclass(frozen=True)
class Result:
    bounding_box: Tensor
    score: Tensor


# XYXY: top left and bottom right corners
# XYWH: top left corner, width and height
# CXCWH: center coordinates, width and height
@dataclass(frozen=True)
class BboxType(Enum):
    XYXY = "xyxy"
    XYWH = "xywh"
    CXCWH = "cxcwh"

    def __str__(self) -> str:
        return super().__str__()

In [ ]:
class Singleton:
    def __init__(self, decorated_class: Any) -> None:
        self._decorated = decorated_class

    def get_instance(self) -> Any:
        """
        Returns the singleton instance. Upon its first call, it creates a
        new instance of the decorated class and calls its `__init__` method.
        On all subsequent calls, the already created instance is returned.
        """
        try:
            return self._instance  # type: ignore
        except AttributeError:
            self._instance = self._decorated()
            return self._instance

    def __call__(self) -> None:
        raise TypeError("Singletons must be accessed through get_instance() method.")

    def __instancecheck__(self, inst: Any) -> bool:
        return isinstance(inst, self._decorated)


# All configurations are stored in a json file and loaded here only once
# All other times the class is called the same instance is returned
@Singleton
class Config:
    def __init__(self) -> None:
        with open(file="config.json", mode="r") as fp:
            cfg: Dict[str, Any] = json.load(fp=fp)
        for k, v in cfg.items():
            setattr(self, k, v)
        # self.__dict__.update(cfg)

Dataset preprocessing

In [ ]:
def get_samples(dir_path: str) -> Tuple[List[Sample], List[Sample], List[Sample]]:
    with open(dir_path + "annotations/instances.json", "r") as inst, open(
        dir_path + "annotations/refs(umd).p", "rb"
    ) as refs:
        instances = json.load(inst)
        references = pickle.load(refs)
    train_samples: List[Sample] = []
    val_samples: List[Sample] = []
    test_samples: List[Sample] = []
    for ref in tqdm(references, desc=f"Processing dataset"):
        image_path = get_image_path(dir_path, ref["image_id"], instances)
        caption = get_caption(ref["sentences"])
        bbox = get_bounding_box(ref["ann_id"], instances)
        split = ref["split"]
        # print(split)
        match split:
            case Split.TRAIN.value:
                train_samples.append(Sample(image_path, caption, bbox))
            case Split.VAL.value:
                val_samples.append(Sample(image_path, caption, bbox))
            case Split.TEST.value:
                test_samples.append(Sample(image_path, caption, bbox))
            case _:
                raise ValueError(f"Invalid split: {split}")
    return train_samples, val_samples, test_samples


def get_image_path(dir_path: str, img_id: int, instances: Dict[str, Any]) -> str:
    image_name = next(
        image["file_name"] for image in instances["images"] if image["id"] == img_id
    )
    path = dir_path + "images/" + image_name
    return path


def get_caption(captions: List[Dict[str, Any]]) -> str:
    longest_caption = captions[0]
    for caption in captions:
        if len(caption["sent"]) > len(longest_caption["sent"]):
            longest_caption = caption
    return longest_caption["sent"]


# Bounding boxed converted to format compatible with yolo or torchvision
def get_bounding_box(ann_id: int, instances: Dict[str, Any]) -> Tensor:
    bbox = next(ann["bbox"] for ann in instances["annotations"] if ann["id"] == ann_id)
    bounding_box: Tensor = tensor([])
    bounding_box = box_convert(
        tensor([bbox]), in_fmt="xywh", out_fmt=BboxType.XYXY.value
    )
    return bounding_box


# If the files already exist, don't preprocess again
def preprocess(in_path: str, out_path: str) -> None:
    if (
        os.path.exists(f"{out_path}train_samples.p")
        and os.path.exists(f"{out_path}val_samples.p")
        and os.path.exists(f"{out_path}test_samples.p")
    ):
        return
    train_samples, val_samples, test_samples = get_samples(in_path)

    pickle.dump(train_samples, open(f"{out_path}train_samples.p", "wb"))

    pickle.dump(val_samples, open(f"{out_path}val_samples.p", "wb"))

    pickle.dump(test_samples, open(f"{out_path}test_samples.p", "wb"))

In [ ]:
# The Dataset contains samples with an image with a bounding box and a caption associated with the bounding box.
class VGDataset(Dataset[Tuple[BatchSample, Tensor]]):
    def __init__(
        self,
        dir_path: str,
        split: Split,
        output_bbox_type: BboxType,
        transform_image: Any = None,
        preprocessed: bool = False,
        preprocessed_path: str = "data/processed/",
    ) -> None:
        super().__init__()
        self.dir_path: str = dir_path
        self.split: Split = split
        self.output_bbox_type: BboxType = output_bbox_type
        self.transform_image: Any = transform_image
        self.device: device = torch.device(
            device="cuda" if torch.cuda.is_available() else "cpu"
        )
        if preprocessed:
            preprocess(dir_path, preprocessed_path)
            with open(
                preprocessed_path + f"{self.split.value}_samples.p", "rb"
            ) as samples:
                self.samples: List[Sample] = pickle.load(samples)
        else:
            self.samples: List[Sample] = self.get_samples()  # type: ignore

    def __len__(self) -> int:
        return len(self.samples)

    def __getitem__(self, ref_id: int) -> Tuple[BatchSample, Tensor]:
        caption: Tensor = clip.tokenize(self.samples[ref_id].caption)

        if self.transform_image is not None:
            image_trans, bbox_trans = self.transform_image(
                Image.open(self.samples[ref_id].image_path),
                self.samples[ref_id].bounding_box,
                device=self.device,
            )
            sample_trans: BatchSample = BatchSample(image_trans, caption).to(
                self.device
            )
            return sample_trans, bbox_trans
        else:
            image: Tensor = read_image(self.samples[ref_id].image_path)
            bbox: Tensor = self.samples[ref_id].bounding_box.to(device=self.device)
            sample: BatchSample = BatchSample(image, caption).to(self.device)
            return sample, bbox

    def get_samples(self) -> List[Sample]:
        with open(self.dir_path + "annotations/instances.json", "r") as inst, open(
            self.dir_path + "annotations/refs(umd).p", "rb"
        ) as refs:
            instances = json.load(inst)
            references = pickle.load(refs)
        samples: List[Sample] = []
        for ref in references:
            if self.split.value == ref["split"]:
                image_path = self.get_image_path(ref["image_id"], instances)
                caption = self.get_caption(ref["sentences"])
                bbox = self.get_bounding_box(ref["ann_id"], instances)
                samples.append(Sample(image_path, caption, bbox))
        return samples

    def get_image_path(self, img_id: int, instances: Dict[str, Any]) -> str:
        image_name = next(
            image["file_name"] for image in instances["images"] if image["id"] == img_id
        )
        path = self.dir_path + "images/" + image_name
        return path

    def get_caption(self, captions: List[Dict[str, Any]]) -> str:
        longest_caption = captions[0]
        for caption in captions:
            if len(caption["sent"]) > len(longest_caption["sent"]):
                longest_caption = caption
        return f"find the region that corresponds to the description {longest_caption['sent']}"

    # Bounding boxed converted to format compatible with yolo or torchvision
    def get_bounding_box(self, ann_id: int, instances: Dict[str, Any]) -> Tensor:
        bbox = next(
            ann["bbox"] for ann in instances["annotations"] if ann["id"] == ann_id
        )
        bounding_box: Tensor = tensor([])
        match self.output_bbox_type:
            case BboxType.XYXY:
                bounding_box = box_convert(
                    tensor([bbox]), in_fmt="xywh", out_fmt=BboxType.XYXY.value
                )
            case BboxType.XYWH:
                bounding_box = box_convert(
                    tensor([bbox]), in_fmt="xywh", out_fmt=BboxType.XYWH.value
                )
            case BboxType.CXCWH:
                bounding_box = box_convert(
                    tensor([bbox]), in_fmt="xywh", out_fmt=BboxType.CXCWH.value
                )

        return bounding_box

Dataset / Dataloader utility functions

In [ ]:
# Function to create the batches
# Each batch is a tuple with a list of BatchSample and a tensor with the bounding boxes
# (since the tensors have all the same size we can stack them in a tensor instead of a list)
def custom_collate(
    batch: List[Tuple[BatchSample, torch.Tensor]]
) -> Tuple[List[BatchSample], torch.Tensor]:
    bboxes: List[torch.Tensor] = []
    samples: List[BatchSample] = []
    for sample, bbox in batch:
        samples.append(BatchSample(sample.image, sample.caption))
        bboxes.append(bbox)
    return samples, torch.stack(bboxes)


# Function to resize the image and the corresponding bounding box correctly
# Bounding box already in the correct format
def transform_sample(
    image: Image.Image,
    box: Tensor,
    target_size: int = 224,
    device: device = torch.device("cpu"),
) -> Tuple[Tensor, Tensor]:
    x: int
    y: int
    x, y = image.size[0], image.size[1]

    x_scale: float = target_size / x
    y_scale: float = target_size / y

    # Same transformation as in the paper
    trans = T.Compose(
        transforms=[
            T.Resize(
                size=target_size,
                interpolation=T.InterpolationMode.BICUBIC,
                max_size=None,
                antialias="warn",
            ),
            T.CenterCrop(target_size),
            T.ToTensor(),
            T.Normalize(
                mean=(0.48145466, 0.4578275, 0.40821073),
                std=(0.26862954, 0.26130258, 0.27577711),
            ),
        ]
    )

    image_tensor: Tensor = trans(image).to(device)  # type: ignore
    if image_tensor.shape[0] == 1:
        image_tensor = image_tensor.repeat(3, 1, 1)

    xmin, ymin, xmax, ymax = box.squeeze(0)

    xmin = np.round(xmin * x_scale)
    ymin = np.round(ymin * y_scale)
    xmax = np.round(xmax * x_scale)
    ymax = np.round(ymax * y_scale)

    bbox_tensor: Tensor = torch.tensor([xmin, ymin, xmax, ymax], device=device)
    return image_tensor, bbox_tensor

In [ ]:
# Simple forward pass on the pretrained CLIP text encoder
class TextEncoder(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.pretrained_model: CLIP = clip.load("RN50", device=self.device)[0]
        self.pretrained_model.float()

    @torch.no_grad()
    def forward(self, tokenized_caption: Tensor) -> Tensor:
        out: Tensor = self.pretrained_model.encode_text(tokenized_caption).to(
            self.device
        )
        return out

In [ ]:
# Class that gets output for all layes of the backbone
# CLIP backbone is a modified ResNet with an attention layer for global pooling
class VisualEncoder(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.pretrained_model: ModifiedResNet = clip.load("RN50", device=self.device)[
            0
        ].visual  # type: ignore
        self.pretrained_model.float()
        assert isinstance(self.pretrained_model, ModifiedResNet)

        # Freeze the backbone
        for param in self.pretrained_model.parameters():
            param.requires_grad = False

        # Register hooks to get the output of all layers
        self.layers_outputs: OrderedDict[str, Tensor] = OrderedDict()
        self.pretrained_model.layer1.register_forward_hook(self.hook_fn("layer1"))  # type: ignore
        self.pretrained_model.layer2.register_forward_hook(self.hook_fn("layer2"))  # type: ignore
        self.pretrained_model.layer3.register_forward_hook(self.hook_fn("layer3"))  # type: ignore
        self.pretrained_model.layer4.register_forward_hook(self.hook_fn("layer4"))  # type: ignore

        # Project the output of each layer to the same dimensionality as the text features
        cfg = Config.get_instance().visual_encoder  # type: ignore
        resnet_resolution: int = cfg["resnet_resolution"]
        resnet_channels: int = cfg["resnet_channels"]

        self.layers_projections: List[nn.Sequential] = []
        for _ in range(4):
            resnet_resolution //= 2
            in_features: int = resnet_channels * resnet_resolution * resnet_resolution
            resnet_channels *= 2
            layer_projection: nn.Sequential = nn.Sequential(
                nn.AdaptiveAvgPool2d(resnet_resolution),
                nn.Flatten(start_dim=1),
                nn.Linear(in_features, cfg["output_dim"], device=self.device),
                nn.ReLU(),
            )
            self.layers_projections.append(layer_projection)

    def forward(self, batch: Tensor) -> OrderedDict[str, Tensor]:
        # Reset the dictionary
        self.layers_outputs = OrderedDict()

        with torch.no_grad():
            out: Tensor = self.pretrained_model(batch)

        for idx, (layer_name, layer_output) in enumerate(self.layers_outputs.items()):
            self.layers_outputs[layer_name] = self.layers_projections[idx](layer_output)
        self.layers_outputs["output"] = out

        return self.layers_outputs

    def hook_fn(self, layer: str) -> Callable[[nn.Module, Tensor, Tensor], None]:
        def hook(module: nn.Module, input: Tensor, output: Tensor) -> None:
            # print(f"Module: {[module for  module in module.modules()]}")
            self.layers_outputs[layer] = output

        return hook

In [ ]:
# The model is composed of the visual encoder and the text encoder described above
# The attention layer is used to compute the attention between the the text and each layer of visual features
# The attentions are then concatenated and passed through a regression head to predict the bounding box
class VGModel(nn.Module):
    def __init__(
        self,
        mlp_hidden_dim_1: int,
        mlp_hidden_dim_2: int,
    ) -> None:
        super().__init__()
        cfg = Config.get_instance().model  # type: ignore
        emb_dim = cfg["emb_dim"]
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.visual_backbone: VisualEncoder = VisualEncoder().to(self.device)
        self.text_encoder: TextEncoder = TextEncoder().to(self.device)
        self.attention_layer = nn.MultiheadAttention(
            embed_dim=emb_dim,
            num_heads=4,
            batch_first=True,
            device=self.device,
        )
        self.reg_head: MLP = MLP(
            emb_dim * 5, 4, hidden_dim_1=mlp_hidden_dim_1, hidden_dim_2=mlp_hidden_dim_2
        ).to(self.device)

    def forward(self, batch: List[BatchSample]) -> Tensor:
        captions: Tensor = torch.stack([sample.caption for sample in batch]).squeeze(1)
        text_features: Tensor = self.text_encoder(captions)

        images: Tensor = torch.stack([sample.image for sample in batch])
        visual_features: OrderedDict[str, Tensor] = self.visual_backbone(images)

        attended_features: List[Tensor] = []
        for feature in visual_features.values():
            attention: Tensor = self.attention_layer(
                feature, text_features, text_features
            )
            attended_features.append(attention[0])

        aggregated_features: Tensor = torch.cat(attended_features, dim=1)
        return self.reg_head(aggregated_features)


# Simple 2-layer MLP
class MLP(nn.Module):
    def __init__(
        self, input_dim: int, output_dim: int, hidden_dim_1: int, hidden_dim_2: int
    ) -> None:
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim_1),
            nn.ReLU(),
            nn.Linear(hidden_dim_1, hidden_dim_2),
            nn.ReLU(),
            nn.Linear(hidden_dim_2, output_dim),
        )

    def forward(self, x: Tensor) -> Tensor:
        return self.mlp(x)

In [ ]:
class Loss:
    def __init__(self, l: float) -> None:
        self.l1_loss = nn.L1Loss(reduction="mean")
        self.giou_loss = generalized_box_iou_loss
        self.l = l
        self.loss: Tensor

    def compute(self, out: Tensor, bbox: Tensor) -> Tensor:
        self.loss = self.giou_loss(out, bbox, reduction="mean") + self.l * self.l1_loss(
            out, bbox
        )
        return self.loss

    def to_float(self) -> float:
        return self.loss.detach().cpu().item()

Main training code

In [ ]:
def train_one_epoch(
    dataloader: DataLoader[Tuple[BatchSample, Tensor]],
    model: VGModel,
    loss: Loss,
    optimizer: torch.optim.Optimizer,
    device: torch.device,
) -> Tensor:
    # As loss we take smooth_l1 + GIoU
    epoch_loss_list: List[Tensor] = []

    for batch, bbox in tqdm(dataloader, desc="Batches"):
        # Move to gpu
        for sample in batch:
            sample.to(device)

        # Forward pass
        out: Tensor = model(batch)

        # Loss and metrics
        batch_loss: Tensor = loss.compute(out, bbox)
        epoch_loss_list.append(batch_loss)

        # Backward pass
        batch_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return torch.stack(epoch_loss_list).mean()


@torch.no_grad()
def validate(
    dataloader: DataLoader[Tuple[BatchSample, Tensor]],
    model: VGModel,
    device: torch.device,
) -> float:
    model.eval()
    # As accuracy we take the average IoU
    accuracy_list: List[Tensor] = []
    for batch, bbox in tqdm(dataloader, desc="Batches"):
        # Move to gpu
        for sample in batch:
            sample.to(device)

        # Forward pass
        out: Tensor = model(batch)

        accuracy_list.append(torch.diagonal(box_iou(out, bbox)).mean())

    return torch.stack(accuracy_list).mean().cpu().item()

In [ ]:
def objective(trial: Trial) -> float:
    cfg = Config.get_instance()  # type: ignore
    train_dataset: VGDataset = VGDataset(
        dir_path=cfg.dataset_path,
        split=Split.TRAIN,
        output_bbox_type=BboxType.XYXY,
        transform_image=transform_sample,
        preprocessed=True,
    )
    print("Train dataset created. Dataset length ", len(train_dataset))

    val_dataset: VGDataset = VGDataset(
        dir_path=cfg.dataset_path,
        split=Split.VAL,
        output_bbox_type=BboxType.XYXY,
        transform_image=transform_sample,
        preprocessed=True,
    )
    print("Validation dataset created. Dataset length: ", len(val_dataset))

    batch_size = trial.suggest_int(
        "batch_size",
        1,
        10,
    )
    train_dataloader: DataLoader[Tuple[BatchSample, Tensor]] = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        collate_fn=custom_collate,
        shuffle=True,
        drop_last=True,
    )

    val_dataloader: DataLoader[Tuple[BatchSample, Tensor]] = DataLoader(
        dataset=val_dataset,
        batch_size=batch_size,
        collate_fn=custom_collate,
        shuffle=True,
        drop_last=True,
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Loss is the weighted sum of the smooth l1 loss and the GIoU
    l = trial.suggest_float("l", 0.0, 1.0)
    loss_func = Loss(l)
    losses_list: List[float] = []
    accuracies_list: List[float] = []

    hidden_dim_1 = trial.suggest_int("hidden_dim_1", 512, 2048)
    hidden_dim_2 = trial.suggest_int("hidden_dim_2", 128, 512)
    if cfg.logging["resume"]:
        checkpoint: Dict[str, Any] = torch.load(cfg.logging["path"] + "model.pth")
        model = VGModel(hidden_dim_1, hidden_dim_2).to(device)
        model.load_state_dict(checkpoint["model_state_dict"])
        lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
        optimizer = optim.AdamW(model.parameters(), lr=lr)
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        lr_scheduler = optim.lr_scheduler.ExponentialLR(
            optimizer, gamma=cfg.model["gamma"]
        )
        lr_scheduler.load_state_dict(checkpoint["lr_scheduler_state_dict"])
        start_epoch: int = checkpoint["epoch"]
        losses_list.append(checkpoint["loss"])
    else:
        model = VGModel(hidden_dim_1, hidden_dim_2).train()
        lr = trial.suggest_float("lr", 1e-5, 1e-2)
        optimizer = optim.AdamW(model.parameters(), lr=lr)
        lr_scheduler = optim.lr_scheduler.ExponentialLR(
            optimizer, gamma=cfg.model["gamma"]
        )
        start_epoch = 0

    for epoch in tqdm(range(start_epoch, cfg.epochs), desc="Epochs"):
        print("-------------------- Training --------------------------")
        epoch_loss = train_one_epoch(
            train_dataloader, model, loss_func, optimizer, device
        )
        losses_list.append(epoch_loss.cpu().item())
        lr_scheduler.step()

        # Evaluate on validation set for hyperparameter tuning
        print("-------------------- Validation ------------------------")
        accuracy = validate(val_dataloader, model, device)
        accuracies_list.append(accuracy)
        trial.report(accuracy, epoch)
        print(f"Accuracy: {accuracy} at epoch {epoch}")

        # Early stopping for non promising trials
        if trial.should_prune():
            raise optuna.TrialPruned()

        # Save model after each epoch
        if cfg.logging["save_model"]:
            dir: str = cfg.logging["path"]
            if not os.path.exists(dir):
                os.makedirs(dir)

            torch.save(
                obj={
                    "epoch": epoch,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "lr_scheduler_state_dict": lr_scheduler.state_dict(),
                    "loss": epoch_loss,
                },
                f=f"{cfg.logging['path']}model.pth",
            )

        torch.clear_autocast_cache()

    return sum(accuracies_list) / len(accuracies_list)


def main() -> None:
    # optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
    study_name = "train.db"
    study = optuna.create_study(
        study_name=study_name,
        storage=f"sqlite:///{study_name}",
        direction="maximize",
        load_if_exists=True,
    )
    study.optimize(objective, n_trials=1, timeout=600)

    trial = study.best_trial
    print(f"Best hyperparameters: {trial.params}")
    fig = plot_optimization_history(study)
    fig.show()

In [ ]:
main()